In [1]:
import pandas as pd
import sqlite3
sqlite_db = 'database.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [3]:
loans = pd.read_csv('loan.csv')
loans.head()

/Users/polbrigneti/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
loans.columns

Index([u'id', u'member_id', u'loan_amnt', u'funded_amnt', u'funded_amnt_inv',
       u'term', u'int_rate', u'installment', u'grade', u'sub_grade',
       u'emp_title', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'issue_d', u'loan_status', u'pymnt_plan',
       u'url', u'desc', u'purpose', u'title', u'zip_code', u'addr_state',
       u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths',
       u'mths_since_last_delinq', u'mths_since_last_record', u'open_acc',
       u'pub_rec', u'revol_bal', u'revol_util', u'total_acc',
       u'initial_list_status', u'out_prncp', u'out_prncp_inv', u'total_pymnt',
       u'total_pymnt_inv', u'total_rec_prncp', u'total_rec_int',
       u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee',
       u'last_pymnt_d', u'last_pymnt_amnt', u'next_pymnt_d',
       u'last_credit_pull_d', u'collections_12_mths_ex_med',
       u'mths_since_last_major_derog', u'policy_code', u'application_type',
       u'annu

In [6]:
# Extracting month, year and loan_term
loans['month'] = loans.issue_d.apply(lambda x: x[:3])
loans['year'] = loans.issue_d.apply(lambda x: x[4:])
loans['loan_term'] = loans.term.apply(lambda x: x[:3])
loans.loan_term = loans.loan_term.astype('int64')

In [ ]:
# Calculating duration of the loans
loans['length_years'] = loans.loan_term/12
loans.year = loans.year.astype(int)
# Calculating end date for each loan
loans['end_d'] = loans.year + loans.length_years
loans['end_date'] = loans.month +"-"+loans.end_d.map(str)

In [8]:
# Function to extract the month number
def month_num(input_str):
    if input_str == 'Jan':
        return '01'
    elif input_str == 'Feb':
        return '02'
    elif input_str == 'Mar':
        return '03'
    elif input_str == 'Apr':
        return '04'
    elif input_str == 'May':
        return '05'
    elif input_str == 'Jun':
        return '06'
    elif input_str == 'Jul':
        return '07'
    elif input_str == 'Aug':
        return '08'
    elif input_str == 'Sep':
        return '09'
    elif input_str == 'Oct':
        return '10'
    elif input_str == 'Nov':
        return '11'
    elif input_str == 'Dec':
        return '12'

In [9]:
# Function to get the quarter
def quarter_num(input_str):
    if input_str == 'Jan':
        return 'Q1'
    elif input_str == 'Feb':
        return 'Q1'
    elif input_str == 'Mar':
        return 'Q1'
    elif input_str == 'Apr':
        return 'Q2'
    elif input_str == 'May':
        return 'Q2'
    elif input_str == 'Jun':
        return 'Q2'
    elif input_str == 'Jul':
        return 'Q3'
    elif input_str == 'Aug':
        return 'Q3'
    elif input_str == 'Sep':
        return 'Q3'
    elif input_str == 'Oct':
        return 'Q4'
    elif input_str == 'Nov':
        return 'Q4'
    elif input_str == 'Dec':
        return 'Q4'

In [10]:
# Adding columns with month number and quarter
loans['month_num'] = loans.month.apply(lambda x: month_num(x))
loans['quarter'] = loans.month.apply(lambda x: quarter_num(x))

In [11]:
# Calculating loans revenue and gmv
loans['loan_gmv'] = loans.loan_term * loans.installment
loans['loan_revenue'] = loans.loan_gmv - loans.loan_amnt

In [57]:
loans_db = loans[['id','member_id','loan_amnt','loan_term','funded_amnt','funded_amnt_inv','grade', 'purpose','sub_grade','addr_state','int_rate','issue_d','installment','end_d','month','term','year','loan_status','month_num','quarter','loan_revenue','loan_gmv','total_pymnt']]
loans_db.to_sql('loans_data', con=engine)

In [43]:
loans.loan_term.value_counts()

36    621125
60    266254
Name: loan_term, dtype: int64

### Distribution of the loans

In [46]:
loan_distribution = engine.execute("""
                                    SELECT
                                        SUM(CASE WHEN loan_amnt <= "2500" THEN 1 ELSE 0 END) AS SIZE1,
                                        SUM(CASE WHEN loan_amnt BETWEEN "2501" AND "5000" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "5001" AND "7500" then 1 ELSE 0 END) AS SIZE3,
                                        SUM(CASE WHEN loan_amnt BETWEEN "7501" AND "10000" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "10001" AND "12500" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "12501" AND "15000" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "15001" AND "17500" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "17501" AND "20000" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "20001" AND "22500" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "22501" AND "25000" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "25001" AND "27500" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "27501" AND "30000" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt BETWEEN "30001" AND "32500" then 1 ELSE 0 END) AS SIZE2,
                                        SUM(CASE WHEN loan_amnt > "32501" THEN 1 ELSE 0 END) AS SIZE1
                                    FROM
                                        loans_data
                                    WHERE
                                        loan_term = "60"
                                        """).fetchall()
loan_distribution_df = pd.DataFrame(data=loan_distribution, columns=['<=2500','2501-5000','5001-7500','7501-10000','10001-12500','12501-15000','15001-17500','17501-20000','20001-22500','22501-25000','25001-27500','27501-30000','30001-32500','>32501'])
loan_distribution_df

,<=2500,2501-5000,5001-7500,7501-10000,10001-12500,12501-15000,15001-17500,17501-20000,20001-22500,22501-25000,25001-27500,27501-30000,30001-32500,>32501
0,377,1659,1561,13167,33089,36778,25189,43308,18878,33637,8270,22165,4890,23286


In [47]:
loan_distribution_df.to_csv('loan_distribution')

### Outstanding loans over time

In [ ]:
outstanding_loans = engine.execute("""SELECT 
                                        year, 
                                        quarter, 
                                        COUNT(id) 
                                    FROM 
                                        loans_data
                                    WHERE 
                                        loan_status <> "Fully paid" 
                                    GROUP BY 
                                        1,2
                                    ORDER BY
                                        1""").fetchall()
outstanding_loans_df = pd.DataFrame(data=outstanding_loans, columns=['year','quarter','loans'])
outstanding_loans_df.to_csv('outstanding_loans')

In [ ]:
amnt_outstanding = engine.execute("""
                                    SELECT 
                                        year, 
                                        quarter, 
                                        SUM(loan_amnt) 
                                    FROM 
                                        loans_data
                                    WHERE 
                                        loan_status <> "Fully paid" 
                                    GROUP BY 
                                        1,2
                                    ORDER BY
                                        1""").fetchall()
amnt_outstanding_df = pd.DataFrame(data=amnt_outstanding, columns=['year','quarter','loans'])
amnt_outstanding_df.to_csv('amnt_outstanding_loans')

### Current status of the loans

In [17]:
loans.loan_status.value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

### Grade classifications of the loans

In [ ]:
# Loans per grade
loans_grade = loans.grade.value_counts()
loans_grade.to_csv('loans_grade')

In [ ]:
loans_grade

In [ ]:
loans.groupby(['grade','sub_grade'])['loan_amnt'].sum()

In [ ]:
# Amount borrowed per grade 
amnt_per_grade = engine.execute("""SELECT 
                                        grade, 
                                        SUM(loan_amnt) 
                                    FROM 
                                        loans_data 
                                    GROUP BY 
                                        1""").fetchall()
amnt_per_grade

In [ ]:
# Loans per sub_grade
loans.sub_grade.value_counts()

In [ ]:
# Amount borrowed per sub_grade 
amnt_per_sub_grade = engine.execute(""""SELECT 
                                            sub_grade, 
                                            SUM(loan_amnt) 
                                        FROM 
                                            loans_data 
                                        GROUP BY 
                                            1""").fetchall()
amnt_per_sub_grade_df = pd.DataFrame(data=amnt_per_sub_grade, columns=['sub_grade','loan_amount'])
amnt_per_sub_grade_df.to_csv('amnt_per_sub_grade')

### Loan Purposes

In [ ]:
loans.purpose.value_counts()

In [ ]:
amnt_loans_purpose = engine.execute("""SELECT 
                                            purpose, 
                                            SUM(loan_amnt) 
                                        FROM 
                                            loans_data 
                                        GROUP BY 
                                            1 
                                        ORDER BY 
                                            2 DESC""").fetchall()
amnt_loans_purpose_df = pd.DataFrame(data=amount_loans_purpose, columns=['purpose','loan_amount'])
amnt_loans_purpose_df.to_csv('amnt_loans_purpose')

### Distribution of loans per state (Volume and Value)

In [ ]:
loan_state_distribution = engine.execute("""SELECT 
                                                addr_state, 
                                                SUM(loan_amnt) AS value, 
                                                COUNT(addr_state) AS volume 
                                            FROM 
                                                loans_data 
                                            GROUP BY 
                                                1""").fetchall()
loan_state_distribution_df = pd.DataFrame(data=loan_state_distribution, columns=['addr_state','value', 'volume'])

In [ ]:
loan_state_distribution_df.to_csv('loan_state_distribution')

### Range of Interest Rates per Grade

In [ ]:
interest_per_grade = engine.execute("""SELECT 
                                            grade, 
                                            int_rate AS interest 
                                        FROM 
                                            loans_data 
                                        GROUP BY 
                                            1,2""").fetchall()
interest_per_grade

In [ ]:
interest_per_grade_df = pd.DataFrame(data=interest_per_grade, columns=['grade','sub_grade','loans','interest'])
interest_per_grade_df.to_csv('interests_per_sub_grade')

In [ ]:
# Calculating interest per grade over time
interest_over_time = engine.execute("""
                                        SELECT
                                            year,
                                            month,
                                            month_num,
                                            ROUND(AVG(int_rate),2)
                                        FROM 
                                            loans_data
                                        WHERE
                                            grade = "G"
                                        GROUP BY
                                            1,2
                                        ORDER BY
                                            1,3,4""").fetchall()

In [ ]:
interest_over_time_df = pd.DataFrame(data=interest_over_time, columns=['year','month','month_num','int_rate'])
interest_over_time_df.to_csv('interst_rate_grade')

# Trends

In [ ]:
amount_loans = engine.execute("""
                                SELECT 
                                    year, 
                                    month, 
                                    month_num, 
                                    COUNT(id) 
                                FROM 
                                    loans_data
                                GROUP BY 
                                    1,2 
                                ORDER BY 
                                    1,3 DESC""").fetchall()

In [ ]:
amount_loans_df = pd.DataFrame(data=amount_loans, columns=['year','month','month_num','loans'])
amount_loans_df.to_csv('amount_loans')

### Loan Performance

In [16]:
performance = engine.execute("""
                                SELECT
                                    grade,
                                    ROUND(SUM(installment),2) as Issued
                                FROM
                                    loans_data
                                GROUP BY
                                    1""").fetchall()
performance

[(u'A', 63371052.95),
 (u'B', 103962094.87),
 (u'C', 103738680.15),
 (u'D', 63571180.63),
 (u'E', 36412625.42),
 (u'F', 13036069.86),
 (u'G', 3441903.88)]

### Growth

In [ ]:
growth = engine.execute("""
                        SELECT
                            year,
                            quarter,
                            COUNT(*),
                            SUM(loan_amnt)
                        FROM
                            loans_data
                        GROUP BY 1,2""").fetchall()
growth_df = pd.DataFrame(data=growth, columns=['year','quarter','loans','amount'])
growth_df.to_csv('growth')

### Profitability

In [ ]:
# Assumption: We include the loans that "Does not meet the credit policy. Status:Fully Paid" as fully paid 
loan_profit = engine.execute("""
                SELECT
                     year,
                    month,
                    month_num,
                    ROUND(SUM(loan_amnt),2) AS amnt_borrowed,
                    ROUND(SUM(loan_gmv),2) AS return
                FROM
                    loans_data
                GROUP BY
                 1,2
                ORDER BY
                 1,3   """).fetchall()

loan_profit_df = pd.DataFrame(data=loan_profit, columns=['year','month','month_num','amnt_borrowed','return'])
loan_profit_df.to_csv('loan_profit')

In [ ]:
loan_profit_df

### Active loans (Value & Volume)

In [ ]:
active_loans = engine.execute("""
                                SELECT
                                    year,
                                    month,
                                    month_num,
                                    COUNT(*) AS loans,
                                    SUM(loan_amnt)
                                FROM
                                    loans_data
                                WHERE
                                    loan_status <> "Fully Paid"
                                GROUP BY
                                    1,2
                                ORDER BY
                                    1,3""").fetchall()
active_loans_df = pd.DataFrame(data=active_loans, columns=['year','month','month_num','volume','value'])
active_loans_df.to_csv('active_loans')

In [ ]:
active_loans_df

In [ ]:
paid_loans = engine.execute("""
                                SELECT
                                    year,
                                    month,
                                    month_num,
                                    SUM(loan_amnt),
                                    ROUND(SUM(loan_gmv),2)
                                FROM
                                    loans_data
                                WHERE
                                    loan_status = "Fully Paid"
                                GROUP BY
                                    1,2
                                ORDER BY
                                    1,3""").fetchall()
paid_loans_df = pd.DataFrame(data=paid_loans, columns=['year','month','month_num','loan_amnt','loan_gmv'])
paid_loans_df.to_csv('paid_loans_amnt')

In [ ]:
loans.loan_status.value_counts()

### Grade distribution per year

In [ ]:
grade_distribution = engine.execute("""
                SELECT
                    year,
                    grade,
                    COUNT(*) as default_loans
                FROM
                    loans_data
                WHERE
                    loan_term = "36"
                GROUP BY
                    1,2""").fetchall()
grade_distribution_df = pd.DataFrame(data = grade_distribution, columns=['year','grade','loans'])
grade_distribution_df.to_csv('grade_distribution')

In [ ]:
grade_distribution_df

In [ ]:
loans.term.value_counts()

### Default rate

In [ ]:
default_rate = engine.execute("""
                SELECT
                    year,
                    month,
                    month_num,
                    SUM(CASE WHEN loan_status IN("Charged Off","Default","Does not meet the credit policy. Status:Charged Off ") THEN 1 ELSE 0 END) as default_loans,
                    COUNT(*) as total_loans
                FROM
                    loans_data
                GROUP BY
                    1,2
                ORDER BY
                    1,3""").fetchall()
default_rate_df = pd.DataFrame(data=default_rate, columns=['year','month','month_num','default_loans','total_loans'])
default_rate_df.to_csv('default_rate')

In [ ]:
default_rate_df

In [ ]:
amnt_default_rate = engine.execute("""
                SELECT
                    year,
                    month,
                    month_num,
                    SUM(CASE WHEN loan_status IN("Charged Off","Default","Does not meet the credit policy. Status:Charged Off ") THEN 1 ELSE 0 END) as default_loans,
                    ROUND(SUM(total_pymnt),2) as volume,
                    SUM(loan_amnt)
                FROM
                    loans_data
                WHERE
                    loan_status IN("Charged Off","Default","Does not meet the credit policy. Status:Charged Off ") AND grade = "A"
                GROUP BY
                    1,2
                ORDER BY
                    1,3""").fetchall()
amnt_default_rate_df = pd.DataFrame(data=amnt_default_rate, columns=['year','month','month_num','default_loans','paid_to_date','total_loans'])
amnt_default_rate_df.to_csv('default_rate')

In [ ]:
amnt_default_rate_df

In [ ]:
grade_loans = engine.execute("""
                SELECT
                    year,
                    month,
                    month_num,
                    SUM(loan_amnt) AS value,
                    ROUND(SUM(loan_gmv),2) AS gmv
                FROM
                    loans_data
                WHERE
                    grade = "G" and loan_status = "Fully Paid"
                GROUP BY
                    1,2
                ORDER BY
                    1,3""").fetchall()
grade_loans_df = pd.DataFrame(data=grade_loans, columns=['year','month','month_num','value','gmv'])
grade_loans_df.to_csv('grade_loans')